In [ ]:
# Impurity Cleanup

import pandas as pd
import re

# Config
pd.set_option('display.max_colwidth', None)
input_path = "CLST.tsv"

# Regex ranges:
DEVANAGARI = r'[\u0900-\u097F]'   # Hindi script
ASSAMESE   = r'[\u0980-\u09FF]'   # Assamese script

patterns = [
    r'\?{2,}',      # multiple question marks
    r'\.{3,}',      # multiple dots
    r'!{2,}',       # multiple exclamation points
    r'<[^>]*>?',    # any tags
    r'�',           # replacement char
]

def has_impurity(text):
    return any(re.search(p, str(text)) for p in patterns)

def clean_text(text):
    out = str(text)
    for p in patterns:
        out = re.sub(p, "", out)
    # Remove leading and trailing double quotes
    out = out.strip().strip('"')
    return out

# LOAD (one‑split to avoid concat issues)
rows = []
with open(input_path, encoding="utf-8") as f:
    for line in f:
        a, b = (line.rstrip("\n").split("\t", 1) + [""])[:2]
        rows.append((a, b))
df = pd.DataFrame(rows, columns=["asm","eng"])
print(f"Loaded {len(df)} rows\n")

# DETECT & CLEAN PUNCTUATION IMPURITIES
mask_impure = df['asm'].apply(has_impurity) | df['eng'].apply(has_impurity)
print(f"Impure rows (by punctuation): {mask_impure.sum()}")

print("\n--- BEFORE cleaning ---")
display(df.loc[mask_impure, ["asm","eng"]])

df.loc[mask_impure, "asm"] = df.loc[mask_impure, "asm"].apply(clean_text)
df.loc[mask_impure, "eng"] = df.loc[mask_impure, "eng"].apply(clean_text)

print("\n--- AFTER cleaning ---")
display(df.loc[mask_impure, ["asm","eng"]])


# SHOW mixed‐script in English (for review only)
mask_mixed_eng = df['eng'].str.contains(ASSAMESE, regex=True) | df['eng'].str.contains(DEVANAGARI, regex=True)
print(f"\nRows where English column contains Assamese or Hindi script: {mask_mixed_eng.sum()}")
display(df.loc[mask_mixed_eng, ["asm","eng"]])

In [ ]:
# Cell 3: Remove Duplicates
# Show exact duplicates (including both source and target)
duplicates = df[df.duplicated()]
print(f"Found {len(duplicates)} exact duplicate rows:")
display(duplicates)

# Save the duplicates for backup or audit if needed
duplicates.to_csv("duplicates_removed.tsv", sep="\t", index=False)

# Remove the exact duplicate rows
df = df.drop_duplicates().reset_index(drop=True)

print(f"\nRemoved {len(duplicates)} duplicate rows. Remaining rows: {len(df)}")


In [ ]:
# Remove rows with NaN in either 'asm' or 'eng' column
df = df.dropna(subset=['asm', 'eng']).reset_index(drop=True)

print(f"After dropping NaN rows: {len(df)} rows")

In [ ]:
# Cell: Remove empty rows

# Remove rows where either column is completely empty or just whitespace
mask_empty = (df['asm'].str.strip() == "") | (df['eng'].str.strip() == "")
print(f"Removing {mask_empty.sum()} rows where either column is empty.")

# Drop them
df = df[~mask_empty].reset_index(drop=True)

print(f"Remaining rows after empty-row removal: {len(df)}")
df.head()

In [ ]:
# Cell 4: Outlier Filtering by Word‐Count Ratio

import re

# 1) Define a word‐count function
def word_count(text):
    # find all alphanumeric “words”
    return len(re.findall(r'\w+', str(text)))

# 2) Compute word counts for source and target
df['words_asm'] = df['asm'].apply(word_count)
df['words_eng'] = df['eng'].apply(word_count)

# 3) Compute ratio
df['ratio'] = df['words_eng'] / df['words_asm'].replace(0, 1)

# 4) Define acceptable ratio range
low, high = 0.2, 5.0
mask_ratio = (df['ratio'] < low) | (df['ratio'] > high)

# 5) Show how many outliers
print("Outlier rows count:", mask_ratio.sum())

# 6) Display the actual sentence pairs that are outliers, with their word counts and ratio
outliers_df = df[mask_ratio]
display(
    outliers_df[[
        'asm',        # Assamese sentence
        'eng',        # English sentence
        'words_asm',
        'words_eng',
        'ratio'
    ]]
)

# 7) Once you’ve inspected them, drop those rows
df = df[~mask_ratio].reset_index(drop=True)
print(f"After ratio filtering: {len(df)} rows remaining")


In [ ]:
# Create length columns
df['len_asm'] = df['asm'].apply(lambda x: len(str(x).split()))
df['len_eng'] = df['eng'].apply(lambda x: len(str(x).split()))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 1) Make sure length columns exist
df['len_asm'] = df['asm'].apply(lambda x: len(str(x).split()))
df['len_eng'] = df['eng'].apply(lambda x: len(str(x).split()))

# 2) Determine the full max length in your data
max_len = max(df['len_asm'].max(), df['len_eng'].max())
print(f"Maximum sentence length in tokens: {max_len}")

# 3) Create bins for every integer length from 0 to max_len
bins = np.arange(0, max_len + 2)

# 4) Decide tick spacing so labels don’t crowd
tick_step = max(1, max_len // 20)
ticks = np.arange(0, max_len + 1, tick_step)

# 5) Plot side-by-side histograms
plt.figure(figsize=(14,5))

# Assamese
plt.subplot(1, 2, 1)
plt.hist(df['len_asm'], bins=bins, edgecolor='black')
plt.title('Assamese Sentence Lengths')
plt.xlabel('Number of Tokens')
plt.ylabel('Number of Sentences')
plt.xticks(ticks, rotation=45)
plt.xlim(0, max_len + 1)

# English
plt.subplot(1, 2, 2)
plt.hist(df['len_eng'], bins=bins, color='orange', edgecolor='black')
plt.title('English Sentence Lengths')
plt.xlabel('Number of Tokens')
plt.ylabel('Number of Sentences')
plt.xticks(ticks, rotation=45)
plt.xlim(0, max_len + 1)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np

while True:
    try:
        min_token_length_str = input("Enter the MINIMUM token length you want to keep for sentences (e.g., 10): ")
        min_token_length = int(min_token_length_str)
        if min_token_length < 0:
            print("Please enter a non-negative integer for minimum length.")
            continue

        max_token_length_str = input("Enter the MAXIMUM token length you want to keep for sentences (e.g., 100): ")
        max_token_length = int(max_token_length_str)
        if max_token_length <= 0:
            print("Please enter a positive integer for maximum length.")
            continue

        if min_token_length > max_token_length:
            print(f"Error: Minimum length ({min_token_length}) cannot be greater than maximum length ({max_token_length}). Please try again.")
        else:
            break
    except ValueError:
        print("Invalid input. Please enter integer numbers for both lengths.")

print(f"\nFiltering sentences to keep those with both Assamese and English lengths between {min_token_length} and {max_token_length} tokens (inclusive).")

keep_mask = (df['len_asm'] >= min_token_length) & \
            (df['len_asm'] <= max_token_length) & \
            (df['len_eng'] >= min_token_length) & \
            (df['len_eng'] <= max_token_length)

# Filtered DataFrame: Contains sentences that meet the length criteria
df_filtered = df[keep_mask].copy()

# Removed DataFrame: Contains sentences that were filtered out (i.e., outside the specified range)
df_removed = df[~keep_mask].copy()

print(f"\nOriginal DataFrame size: {len(df)} rows")
print(f"Filtered DataFrame size (sentences kept within range): {len(df_filtered)} rows")
print(f"Removed DataFrame size (sentences outside range): {len(df_removed)} rows")

# --- Save to TSV files ---
output_filtered_file = f"filtered_dataset_from_{min_token_length}_to_{max_token_length}_tokens.tsv"
output_removed_file = f"removed_dataset_outside_{min_token_length}_to_{max_token_length}_tokens.tsv"
output_full_dataset_file = "complete_dataset_with_token_counts.tsv" # This file remains the same

# --- Prepare DataFrames for Saving with Descriptive Headers ---

# For the filtered and removed datasets (only 'asm' and 'eng' columns)
df_filtered_output = df_filtered[['asm', 'eng']].rename(columns={
    'asm': 'Assamese Sentence',
    'eng': 'English Sentence'
})
df_removed_output = df_removed[['asm', 'eng']].rename(columns={
    'asm': 'Assamese Sentence',
    'eng': 'English Sentence'
})

# For the complete dataset (including token counts)
df_complete_output = df[['asm', 'eng', 'len_asm', 'len_eng']].rename(columns={
    'asm': 'Assamese Sentence',
    'eng': 'English Sentence',
    'len_asm': 'Assamese Token Count',
    'len_eng': 'English Token Count'
})

# Save the filtered data (sentences within range)
df_filtered_output.to_csv(output_filtered_file, sep='\t', index=False, header=True)
print(f"\nSaved filtered data (within token range) to: {output_filtered_file}")

# Save the removed data (sentences outside range)
df_removed_output.to_csv(output_removed_file, sep='\t', index=False, header=True)
print(f"Saved removed data (outside token range) to: {output_removed_file}")

# Save the complete dataset with token counts
df_complete_output.to_csv(output_full_dataset_file, sep='\t', index=False, header=True)
print(f"Saved complete dataset with token counts to: {output_full_dataset_file}")

print("\nFiltering and saving complete.")